In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geoip2.database

%matplotlib inline

In [2]:
reader = geoip2.database.Reader('../data/GeoLite2-City_20181009/GeoLite2-City.mmdb')
#reader.close()


In [4]:
iis_logs_df = pd.read_csv("../data/logs/u_ex181010.log"
            #,skiprows=[0,1,2,3]
            , comment='#'
            , sep=' ' 
            , usecols=[0,1, 2, 5, 6, 7, 8, 9, 10,11,12,14]
            , na_values='-'
            , names=['date'
                    ,'time'
                    ,'server-ip'
                    ,'cs-uri-query'
                    ,'server-port'
                    ,'cs-username'
                    ,'client-ip'
                    ,'cs(User-Agent)'
                    ,'cs(Referer)'
                    ,'sc-status'
                    ,'sc-substatus'
                   ,'time-taken(ms)'])

In [5]:
iis_logs_df.head()

,date,time,server-ip,cs-uri-query,server-port,cs-username,client-ip,cs(User-Agent),cs(Referer),sc-status,sc-substatus,time-taken(ms)
0,2018-10-10,00:00:19,192.168.2.210,NaN,80,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,301,0,93
1,2018-10-10,00:00:19,192.168.2.210,NaN,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,302,0,93
2,2018-10-10,00:00:19,192.168.2.210,ReturnUrl=%2f,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,302,0,93
3,2018-10-10,00:00:19,192.168.2.210,ReturnUrl=%2f,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,200,0,171
4,2018-10-10,00:00:20,192.168.2.210,v=pIhLJRZ-i9xD3brYZ6I5pZD6973WqBDt6kYnAg5j20E1,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,https://inplace.canberra.edu.au/Security/Accou...,200,0,265


In [6]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12334 entries, 0 to 12333
Data columns (total 12 columns):
date              12334 non-null object
time              12334 non-null object
server-ip         12334 non-null object
cs-uri-query      5510 non-null object
server-port       12334 non-null int64
cs-username       7275 non-null object
client-ip         12334 non-null object
cs(User-Agent)    12138 non-null object
cs(Referer)       9206 non-null object
sc-status         12334 non-null int64
sc-substatus      12334 non-null int64
time-taken(ms)    12334 non-null int64
dtypes: int64(4), object(8)
memory usage: 1.1+ MB


In [7]:
iis_logs_df.describe()

,server-port,sc-status,sc-substatus,time-taken(ms)
count,12334.000000,12334.000000,12334.0,12334.000000
mean,442.323091,215.232447,0.0,532.415518
std,15.661410,38.194092,0.0,2264.506444
min,80.000000,200.000000,0.0,0.000000
25%,443.000000,200.000000,0.0,62.000000
50%,443.000000,200.000000,0.0,187.000000
75%,443.000000,200.000000,0.0,330.000000
max,443.000000,404.000000,0.0,82662.000000


In [8]:
iis_logs_df["server-port"].describe()

count    12334.000000
mean       442.323091
std         15.661410
min         80.000000
25%        443.000000
50%        443.000000
75%        443.000000
max        443.000000
Name: server-port, dtype: float64

In [54]:
#Enhance dataset with Client city and country , derived from IP
iis_logs_df['client-city'] =  iis_logs_df['client-ip'].apply(lambda x: reader.city(ip_address=x).city.name if reader.city(ip_address=x).city.name != None else np.nan)
iis_logs_df['client-country'] =  iis_logs_df['client-ip'].apply(lambda x: reader.city(ip_address=x).country.name if reader.city(ip_address=x).country.name != None else np.nan)

In [57]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12334 entries, 0 to 12333
Data columns (total 14 columns):
date              12334 non-null object
time              12334 non-null object
server-ip         12334 non-null object
cs-uri-query      5510 non-null object
server-port       12334 non-null int64
cs-username       7275 non-null object
client-ip         12334 non-null object
cs(User-Agent)    12138 non-null object
cs(Referer)       9206 non-null object
sc-status         12334 non-null int64
sc-substatus      12334 non-null int64
time-taken(ms)    12334 non-null int64
client-city       12208 non-null object
client-country    12334 non-null object
dtypes: int64(4), object(10)
memory usage: 1.3+ MB


In [63]:
iis_logs_df.loc[:, iis_logs_df.dtypes == object].describe()

,date,time,server-ip,cs-uri-query,cs-username,client-ip,cs(User-Agent),cs(Referer),client-city,client-country
count,12334,12334,12334,5510,7275,12334,12138,9206,12208,12334
unique,1,6220,1,2716,84,222,90,313,66,14
top,2018-10-10,02:56:42,192.168.2.210,ReturnUrl=%2f,s437875,137.92.20.64,Mozilla/5.0+(Windows+NT+10.0;+WOW64)+AppleWebK...,https://inplace.canberra.edu.au/Security/Accou...,Canberra,Australia
freq,12334,27,12334,201,1102,993,2289,1242,4119,10003
